In [1]:
from collections import defaultdict
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from consent.consistency.comply_util import site_to_contras, get_comply
from consent.consistency.cookie_pref_match import cookie_pref_match
from consent.consistency.util import get_scan_root_dir, get_scan_dirs # , FIG_DIR uncomment to save fig.
from consent.data.pref_menu_scan.cookie_pref_reader import read_cookie_prefs_in_scans
from consent.data.pref_menu_scan.har_cookie_reader import read_postrej_sent_cookies_in_scans

cur_set = '0k_20k' # '20k_100k'
overwrite = True
SCAN_ROOT_DIR = get_scan_root_dir('sydney')
SCAN_DIRS = [SCAN_ROOT_DIR / f'pref_menu_scan_{cur_set}']
# SCAN_DIRS = get_scan_dirs('us')[:]
output_suffix = '_' + cur_set # '0k_20k'  # '20k_100k' #'60k_80k' # '40k_60k' # # '100k_200k'; done: '20k_40k' '0k_20k'
print('Scan root dir:', SCAN_ROOT_DIR)
print('Output suffix:', output_suffix)
print('Scan dirs:', SCAN_DIRS)

Scan root dir: /mnt/sdd/ducbui/projects/data2/consent/2024-10-07
Output suffix: _0k_20k
Scan dirs: [PosixPath('/mnt/sdd/ducbui/projects/data2/consent/2024-10-07/pref_menu_scan_0k_20k')]


In [2]:
# Get cookie prefs = cookie decls + prefs
cookie_prefs_file = SCAN_ROOT_DIR / f'cookie_prefs{output_suffix}.parquet'

if not cookie_prefs_file.exists() or overwrite:
    save_cookie_decls_file = SCAN_ROOT_DIR / f'cookie_decls{output_suffix}.parquet'
    print(f'To save to {save_cookie_decls_file}')
    cookie_prefs = read_cookie_prefs_in_scans(SCAN_DIRS, save_cookie_decls_file)  # took 2.3min for top 50k-site scan
    cookie_prefs.to_parquet(cookie_prefs_file); print(f'Written to {cookie_prefs_file}')
else:
    cookie_prefs = pd.read_parquet(cookie_prefs_file)
cookie_prefs.head()
# TODO: add log_file_checker

To save to /mnt/sdd/ducbui/projects/data2/consent/2024-10-07/cookie_decls_0k_20k.parquet
https://cdn.cookielaw.org/consent/03b96273-0ebc-43cb-8885-cdfaeadad5b9/b019db6d-bccf-46f9-bde0-478e3c1f6028/en.json WARNING: Cookie list has session cookie but duration > 0

File iberia.com consent_resources.json has no dataWARNING: error decoding /mnt/sdd/ducbui/projects/data2/consent/2024-10-07/pref_menu_scan_0k_20k/scribbr.com/consent_resources.json: region_block Received an out-of-region response
https://cdn-ukwest.onetrust.com/consent/86f42b6d-7a31-469c-804c-319643173d42/018dfc06-4525-75f5-b4f2-881e5366447a/en.json WARNING: Cookie list has session cookie but duration > 0

Written to /mnt/sdd/ducbui/projects/data2/consent/2024-10-07/cookie_decls_0k_20k.parquet
Written to /mnt/sdd/ducbui/projects/data2/consent/2024-10-07/cookie_prefs_0k_20k.parquet


,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
0,.AxcmXFaxAuth,fax.nextiva.com,0 days,C0001,Strictly Necessary Cookies,always active,nextiva.com,onetrust,en.json,True
1,__cf_bm,nextiva.com,0 days,C0001,Strictly Necessary Cookies,always active,nextiva.com,onetrust,en.json,True
2,_ce.cch,nextiva.com,Session,C0001,Strictly Necessary Cookies,always active,nextiva.com,onetrust,en.json,True
3,ao,sso.nextiva.com,0 days,C0001,Strictly Necessary Cookies,always active,nextiva.com,onetrust,en.json,True
4,autolaunch_triggered,sso.nextiva.com,0 days,C0001,Strictly Necessary Cookies,always active,nextiva.com,onetrust,en.json,True


In [3]:
print("Found cookie libraries:")
cookie_prefs[['site', 'lib_name']].drop_duplicates().lib_name.value_counts()

Found cookie libraries:


lib_name
onetrust     1203
cookiebot     100
Name: count, dtype: int64

In [ ]:
cookielist_sites = sorted(cookie_prefs.site.unique().tolist())
n_cookielist_sites = len(cookielist_sites)
assert cookie_prefs.site.nunique() == n_cookielist_sites
n=10
print("Sites with cookie prefs:", n_cookielist_sites, f', first {n} sites:' , cookielist_sites[:n])

Sites with cookie prefs: 1303 , first 10 sites: ['247sports.com', '2k.com', '2ksports.com', 'aber.ac.uk', 'acast.com', 'accedo.tv', 'accenture.com', 'accor.com', 'acehardware.com', 'acm.org']


In [5]:
site_to_libname = {row['site']: row['lib_name'] for row in cookie_prefs[['site', 'lib_name']].drop_duplicates().to_dict('records')}

In [6]:
# Quick checking the cookie declaration naming
print(len(cookie_prefs[cookie_prefs.name.str.endswith('#')]) / len(cookie_prefs))
print(len(cookie_prefs[cookie_prefs.name.str.endswith('xxx')]) / len(cookie_prefs))
print(len(cookie_prefs[cookie_prefs.name.str.endswith('XXX')]) / len(cookie_prefs))
cookie_prefs[cookie_prefs.name.str.endswith('#')]

0.011962376083053625


0.0294278354539068
0.0027866677074389196


,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
101,visitor_id#####,go.nextiva.com,364 days,C0004,Targeting Cookies,active,nextiva.com,onetrust,en.json,False
24,_gd#############,j.6sc.co,730 days,C0002,Performance Cookies,active,wework.com,onetrust,en.json,False
30,_gd#############,www.wework.com,730 days,C0003,Functional Cookies,active,wework.com,onetrust,en.json,False
34,_gd#############,cdnjs.cloudflare.com,730 days,C0003,Functional Cookies,active,wework.com,onetrust,en.json,False
44,AMP_unsent_#,commercial.cdn.aws.placeit.net,Persistent,statistics,statistics,None,placeit.net,cookiebot,cc.js,False
...,...,...,...,...,...,...,...,...,...,...
140,visitor_id#####,click.outreach.io,3649 days,4,Targeting Cookies,inactive,outreach.io,onetrust,en.json,False
147,visitor_id#####,pi.pardot.com,3649 days,4,Targeting Cookies,inactive,outreach.io,onetrust,en.json,False
170,visitor_id#####,pardot.com,3649 days,4,Targeting Cookies,inactive,outreach.io,onetrust,en.json,False
172,_gd#############,j.6sc.co,729 days,4,Targeting Cookies,inactive,outreach.io,onetrust,en.json,False


In [7]:
# # Distribution of errors
# print("Libs in failed-to-crawl sites")
# errors.consent_lib_name.value_counts()
# from ooutil.df_util import expand_dict_col
# ot_errs = errors[errors.consent_lib_name == 'onetrust']
# errors_dict = pd.DataFrame(ot_errs.error.tolist()).explode('reason')
# # Uncomment to show
# # errors_dict.reason.value_counts()

# Main analysis

In [8]:
# import sys; import importlib; importlib.reload(sys.modules['consent.data.pref_menu_scan.postrej_cookie_reader'])
from consent.data.pref_menu_scan.har_cookie_reader import read_postrej_sent_cookies_in_scans

cookies_cache_file = SCAN_ROOT_DIR / f'scan{output_suffix}.parquet'  # 'raw_postrej_sent_cookies.parquet'

if not cookies_cache_file.exists() or overwrite:
    sent_cookies = read_postrej_sent_cookies_in_scans(SCAN_DIRS)
    if cookies_cache_file: sent_cookies.to_parquet(cookies_cache_file); print(f"Written to {cookies_cache_file}")
else:
    sent_cookies = pd.read_parquet(cookies_cache_file)

print(f"Number sent cookies read: {len(sent_cookies):,d}")
sent_cookies.head(3)

Post rej cookie not contain consent cookie, renfe.com/postrej_0.har.xz not have cookie OptanonConsent of .renfe.com on path /
Post rej cookie not contain consent cookie, skrill.com/postrej_0.har.xz not have cookie OptanonConsent of .skrill.com on path /
Unmatch url and consent cookie domain: page_url='https://documentation.solarwinds.com/en/success_center/loggly/content/admin/cloudtrail-log-ingestion.htm?cshid=loggly_cloudtrail-log-ingestion' consent_cookie.cookie['domain']='.www.loggly.com' consent_cookie.cookie['path']='/' loggly.com/postrej_5.har.xz
Unmatch url and consent cookie domain: page_url='https://documentation.solarwinds.com/en/success_center/loggly/content/admin/cloudwatch-logs.htm?cshid=loggly_cloudwatch-logs' consent_cookie.cookie['domain']='.www.loggly.com' consent_cookie.cookie['path']='/' loggly.com/postrej_3.har.xz
Post rej cookie not contain consent cookie, lidl.com/postrej_0.har.xz not have cookie OptanonConsent of .lidl.com on path /
Post rej cookie not contain co

,name,value,domain,path,expires,size,httpOnly,secure,session,sameSite,priority,sameParty,sourceScheme,sourcePort,request_url,site,page_url,partitionKey
0,hubspotutk,1f301d54f6d913727db8d125efa68643,.nextiva.com,/,1.743872e+09,42,False,False,False,Lax,Medium,False,Secure,443,https://www.nextiva.com/cdn-cgi/image/width=36...,nextiva.com,https://www.nextiva.com/products/business-voice,NaN
1,_fbp,fb.1.1728319769812.257900613385585207,.nextiva.com,/,1.736096e+09,41,False,False,False,Lax,Medium,False,Secure,443,https://www.nextiva.com/cdn-cgi/image/width=13...,nextiva.com,https://www.nextiva.com/products/business-voice,NaN
2,_vwo_uuid_v2,D0C5B26E49D73A48E2A33181309B83306|61a17e83da68...,.nextiva.com,/,1.759942e+09,78,False,False,False,NaN,Medium,False,Secure,443,https://www.nextiva.com/cdn-cgi/image/width=10...,nextiva.com,https://www.nextiva.com/products/business-voice,NaN


In [9]:
# prj = post-rejection
# Old way: consider all flows
# prj_sent_cookies = sent_cookies[['domain', 'expires', 'name', 'path', 'sameSite', 'secure', 'value', 'request_url', 'site']].drop_duplicates()
# prj_br_cookies = prj_sent_cookies[['domain', 'expires', 'name', 'path', 'sameSite', 'secure', 'site']].drop_duplicates()
# New way: consider cookies only (ignore the request url, which is not nused in get_comply cookie matching anyway)
prj_sent_cookies = sent_cookies[['name', 'domain', 'path', 'site']].drop_duplicates()

In [10]:
# print(f"Num captured sent cookies: {len(sent_cookies):,d}")
# print(f"Num unique captured cookies: {len(prj_sent_cookies):,d}")
# n_br_cookies = len(prj_br_cookies)
# print(f"Num unique browser cookies: {n_br_cookies:,d} on {prj_br_cookies.site.nunique():,d} websites") # and {sent_cookies.page_url.nunique():,d} pages")

## Analyze cookies compliance    

In [11]:
sites = set(cookie_prefs.site)
print(f"Num sent cookies in the cookie prefs sites: {len(prj_sent_cookies[prj_sent_cookies.site.isin(sites)]):,d}")

Num sent cookies in the cookie prefs sites: 51,509


In [12]:
import sys; import importlib; importlib.reload(sys.modules['consent.consistency.comply_util'])
from consent.consistency.comply_util import get_comply

print("faster: for cookie_pref in cookie_pref_set[cookie_pref_set.name == acookie['name']]:")
all_complies = get_comply(cookie_prefs, prj_sent_cookies)
all_complies.head()

faster: for cookie_pref in cookie_pref_set[cookie_pref_set.name == acookie['name']]:


  0%|          | 0/1303 [00:00<?, ?it/s]

 89%|████████▉ | 1157/1303 [03:16<00:06, 20.88it/s]

 77%|███████▋  | 1006/1303 [03:00<00:23, 12.88it/s]

Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='_px3' bad character range d-4 at position 20
Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='_gid' bad character range d-4 at position 20
Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='_gcl_au' bad character range d-4 at position 20
Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='_ga_TNYBCQTJ6G' bad character range d-4 at position 20
Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='visitor-id' bad character range d-4 at position 20
Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='pxcts' bad character range d-4 at position 20
Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='_ga_64H4Z6TSBF' bad character range d-4 at position 20
Error fuzzy name match pr

100%|██████████| 1303/1303 [03:32<00:00,  6.13it/s]


,name,domain,path,site,comply
0,hubspotutk,.nextiva.com,/,nextiva.com,omit
1,_fbp,.nextiva.com,/,nextiva.com,incorrect
2,_vwo_uuid_v2,.nextiva.com,/,nextiva.com,incorrect
3,pscd,.nextiva.com,/,nextiva.com,omit
4,_vwo_uuid,.nextiva.com,/,nextiva.com,incorrect


In [13]:
# Analyze complies here
complies = all_complies[['name', 'domain', 'path', 'site', 'comply']].drop_duplicates()
comply_sites_data = defaultdict(list)
for comply_type, comply_group in complies.groupby('comply'):
    n_sites = comply_group.site.nunique()
    comply_sites_data['comply_type'].append(comply_type)
    comply_sites_data['num_sites'].append(n_sites)
    comply_sites = pd.DataFrame(comply_sites_data).sort_values(by=['num_sites'], ascending=False)

nsites = all_complies.site.nunique()
comply_counts = complies.comply.value_counts()
comply_sites['num_sites_percent'] = comply_sites['num_sites'] / nsites * 100
comply_sites['num_cookies'] = comply_sites['comply_type'].map(comply_counts)
comply_sites['num_cookies_percent'] = comply_sites['num_cookies'] / comply_sites['num_cookies'].sum() * 100  # need to read scan_*.parquet for n_br_cookies, but may be unnecessary
comply_sites['num_cookies_per_site'] = comply_sites['num_cookies'] / comply_sites['num_sites']

# Do not count comply/correct enforcement: which require detecting all possible
comply_sites = comply_sites[comply_sites.comply_type != 'comply']
comply_sites

,comply_type,num_sites,num_sites_percent,num_cookies,num_cookies_percent,num_cookies_per_site
3,omit,1197,92.076923,25459,49.426314,21.269006
2,incorrect,1014,78.000000,13988,27.156419,13.794872
0,ambiguous,51,3.923077,60,0.116484,1.176471


In [14]:
# Record the analysis results
all_complies_file = SCAN_ROOT_DIR / f'all_complies{output_suffix}.parquet'
all_complies.to_parquet(all_complies_file); print(f"Written {len(all_complies):,d} records of all_complies to {all_complies_file}")

Written 51,509 records of all_complies to /mnt/sdd/ducbui/projects/data2/consent/2024-10-07/all_complies_0k_20k.parquet


In [15]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

raise StopExecution
import sys; sys.exit(0)

In [16]:
complies.comply

0        incorrect
1           comply
2             omit
3           comply
4             omit
           ...    
49292         omit
49293         omit
49294         omit
49295         omit
49296         omit
Name: comply, Length: 49297, dtype: object

In [17]:
nsites = all_complies.site.nunique(); nsites
# all_complies.site.unique()

1279

In [18]:
detected_contra_sites = [s for s, contras in site_to_contras.items() if len(contras) > 0]; len(detected_contra_sites)

0

In [19]:
complies = all_complies[ ['name', 'domain', 'site', 'comply',]].drop_duplicates()
comply_counts = complies.comply.value_counts()
comply_counts

omit         24866
incorrect    12965
comply       11215
ambiguous       94
Name: comply, dtype: int64

In [20]:
complies_sites = set(all_complies.site)
sent_cookies_com = sent_cookies[sent_cookies.site.isin(complies_sites)]
prj_sent_cookies_com = sent_cookies_com[['domain', 'expires', 'name', 'path', 'sameSite', 'secure', 'value', 'request_url', 'site']].drop_duplicates()
prj_br_cookies_com = prj_sent_cookies_com[['domain', 'expires', 'name', 'path', 'sameSite', 'secure', 'site']].drop_duplicates()
print(f"Num captured sent cookies: {len(sent_cookies_com):,d}")
print(f"Num unique captured cookies: {len(prj_sent_cookies_com):,d}")

n_br_cookies_com = len(prj_br_cookies_com)
print(f"Num unique browser cookies: {n_br_cookies_com:,d} on {prj_br_cookies_com.site.nunique():,d} websites") # and {sent_cookies.page_url.nunique():,d} pages")

Num captured sent cookies: 5,628,874
Num unique captured cookies: 3,360,899
Num unique browser cookies: 127,868 on 1,279 websites


# Analyze contras

In [21]:
# Way 1: compute contra sites by dynamic analysis: this should be lower than statically analyzing prefs
# because we cannot check all combinations of consent modes.
from consent.consistency.comply_util import get_appr_rej_sets
contra_data = []
for asite in cookie_prefs.site.unique():
    site_prefs = cookie_prefs[cookie_prefs.site == asite]
    _, _, contras = get_appr_rej_sets(site_prefs)
    for contra in contras:
        contra['site'] = asite
        contra_data.append(contra)
contra_sites = pd.DataFrame(contra_data)
contra_sites.head()

,domain,name,site
0,blog.lumen.com,PHPSESSID,savvis.net
1,news.lumen.com,__cfruid,savvis.net
2,www.menshealth.com,location_data,menshealth.com
3,www.elle.com,location_data,elle.com
4,www.hottopic.com,__cqact,hottopic.com


In [22]:
assert set(detected_contra_sites) == set(contra_sites.site), "this is the same method with above, so the result"

AssertionError: this is the same method with above, so the result

In [ ]:
set(detected_contra_sites) - set(contra_sites.site)
set(contra_sites.site) - set(detected_contra_sites)
# This may be the sites excluded due to testing cookies in the preferences.

{'247sports.com',
 'abbvie.com',
 'acquia.com',
 'adobe.io',
 'adweek.com',
 'aljazeera.com',
 'allaboutvision.com',
 'allegiantair.com',
 'allergan.com',
 'alteryx.com',
 'americanbar.org',
 'aon.com',
 'ariel.co.uk',
 'asos.com',
 'autosport.com',
 'avalara.com',
 'avast.com',
 'avastbrowser.com',
 'barnesandnoble.com',
 'bazaarvoice.com',
 'bbva.com',
 'bd.com',
 'behance.net',
 'belkin.com',
 'betsson.com',
 'bitdefender.com',
 'bitdefender.net',
 'bkstr.com',
 'blackboard.com',
 'bplans.com',
 'brightside.me',
 'bromium-online.com',
 'callofduty.com',
 'cambridge.org',
 'carbonite.com',
 'cheapoair.com',
 'chrysler.com',
 'cisco.com',
 'citationmachine.net',
 'clickmeeting.com',
 'clicktale.net',
 'cloudbees.com',
 'cnet.com',
 'colt.net',
 'commonsensemedia.org',
 'contentsquare.com',
 'conviva.com',
 'creighton.edu',
 'curalate.com',
 'dc.com',
 'dccomics.com',
 'discogs.com',
 'dynamicyield.com',
 'easybib.com',
 'eidos.com',
 'elderscrollsonline.com',
 'evri.com',
 'ew.com',
 

In [ ]:
# Way 2: compute contra sites by DataFrame group by
contra_cookies_dfs = []
for _, same_cookies in cookie_prefs.groupby(['name', 'domain', 'site']):
    consent_modes = same_cookies.consent_mode.unique()
    if len(consent_modes) >= 2 and same_cookies.category.nunique() > 1: # and 'always active' in consent_modes:
        contra_cookies_dfs.append(same_cookies)
contra_cookies = pd.concat(contra_cookies_dfs).drop_duplicates()

In [ ]:
print("Number sites with contra cookies:", contra_cookies.site.nunique())

Number sites with contra cookies: 184


In [ ]:
# set(contra_cookies.site) - set(contra_sites.site)

In [ ]:
print("Num contra cookie settings (double categories):", len(contra_cookies), f"settings  on {contra_cookies.site.nunique()} sites")
print("Contras with 1 always-active:", contra_sites.site.nunique())
print("Contras with both editable:", len(set(contra_cookies.site) - set(contra_sites.site)))

Num contra cookie settings (double categories): 648 settings  on 184 sites
Contras with 1 always-active: 192
Contras with both editable: 0


In [ ]:
contra_cookies[contra_cookies.site == 'elsevier.nl']

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent


In [ ]:
# assert len(detected_contra_sites) <= contra_sites.site.nunique() < len(detected_contra_sites) + 2

In [ ]:
complies[complies.comply == 'ambiguous']

,name,domain,site,comply
173,TS013559a7,www.mastercard.us,mastercard.com,ambiguous
1236,_qubitTracker,.www.very.co.uk,very.co.uk,ambiguous
2365,G_ENABLED_IDPS,.www.glassdoor.com,glassdoor.com,ambiguous
2372,trs,www.glassdoor.com,glassdoor.com,ambiguous
2375,_gid,.glassdoor.com,glassdoor.com,ambiguous
...,...,...,...,...
59468,ak_bmsc,.mulesoft.com,mulesoft.com,ambiguous
59481,JSESSIONID,training.mulesoft.com,mulesoft.com,ambiguous
60397,TLTSID,.bkstr.com,bkstr.com,ambiguous
60412,dtSa,.bkstr.com,bkstr.com,ambiguous


In [ ]:
cookie_prefs[ (cookie_prefs.site == 'michaels.com') & (cookie_prefs.name == 'utag_main') ]

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
88,utag_main,.michaels.com,365 days,C0001,Strictly Necessary Cookies,always active,michaels.com,onetrust,en.json,True
148,utag_main,michaels.com,365 days,C0002,Performance Cookies,active,michaels.com,onetrust,en.json,False


In [ ]:
cookie_prefs[ (cookie_prefs.site == 'statista.com') & (cookie_prefs.name == 'id') ]

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
74,id,m6r.eu,90 days,C0001,Necessary Cookies,always active,statista.com,onetrust,en.json,True
131,id,m6r.eu,0 days,C0004,Targeting Cookies,inactive,statista.com,onetrust,en.json,False


In [ ]:
# contra_cookies.name.value_counts()[:10]
# contra_cookies[contra_cookies.name == '__cf_bm'].reset_index(drop=True) # .site.nunique()

# Analyze ambiguous flows

In [ ]:
ambi_cookies = complies[complies.comply == 'ambiguous']
print("Num ambiguous cookies:", len(ambi_cookies), "on num sites:", ambi_cookies.site.nunique())
assert len(ambi_cookies) == len(ambi_cookies.drop_duplicates()), "there is some duplicate cookies"


## Analyze no-way-to-opt-out

In [ ]:
# Way 1: compute contra sites by DataFrame group by
nwtoo_cookies_dfs = []
for _, site_cookies in cookie_prefs.groupby(['site']):
    consent_modes = site_cookies.consent_mode.unique().tolist()
    if len(consent_modes) == 1 and consent_modes[0] == 'always active':
        nwtoo_cookies_dfs.append(site_cookies)
nwtoo_cookies = pd.concat(nwtoo_cookies_dfs).drop_duplicates()
nwtoo_cookies.head()

In [ ]:
nwtoo_cookies.site.nunique()

# Analyze inconsistent flows/cookies

In [ ]:
def plot_inconsis_types():
    sns.set(rc={'figure.figsize':(9,9),"font.size":30,"axes.titlesize":30,"axes.labelsize":30},style="white", font_scale = 2.5)

    ax = sns.barplot(x='comply_type', y='num_sites_percent', data=noncomply_sites)
    ax.tick_params(left=True, bottom=True, direction="in")
    ax.set_ylabel('# Websites (%)')
    ax.set_xlabel('Noncompliance Types')
    xticks = [x.get_text().title() for x in ax.get_xticklabels()]
    ax.set(xticklabels=xticks)

    out_file = FIG_DIR / 'noncomply_types_per_site.png'
    # plt.savefig(out_file, dpi=200, bbox_inches='tight'); print(f"Written to {out_file}")

In [ ]:
## Plot distribution of omitted cookie declarations

In [ ]:
omit_complies = complies[(complies.comply == 'omit')] #  & ~complies.name.isin(['OptanonAlertBoxClosed', 'OptanonConsent']) ]
omit_complies.site.nunique()

In [ ]:
omit_complies.site.value_counts()[:7]

In [ ]:
omit_cookies = omit_complies.name.value_counts()[:5].to_frame().reset_index().rename(columns={'index': 'cookie_name'})
omit_cookies['num_sites'] = omit_cookies['cookie_name'].map(lambda cookie_name: omit_complies[omit_complies.name == cookie_name].site.nunique())
print(omit_cookies.head(5)[['cookie_name', 'num_sites']].rename(columns={'cookie_name': 'Cookie Name', 'num_sites': '# Websites'}).to_latex(index=False))

In [ ]:
# set(cookielist_sites) - set(omit_complies.site.unique())
omit_counts = omit_complies.site.value_counts().to_frame().rename(columns={'site': 'num_omit_cookies'})
print("Num omitted sites:", omit_complies.site.nunique())
omit_counts.num_omit_cookies.mean(), omit_counts.num_omit_cookies.std(), omit_counts.num_omit_cookies.min(), omit_counts.num_omit_cookies.max()

In [ ]:
sns.set(rc={'figure.figsize':(12,8),"font.size":35,"axes.titlesize":30,"axes.labelsize":50},style="white", font_scale = 4.5)
sns.set_style("ticks", {"xtick.direction": "in", "ytick.direction": "in"})
plt.rcParams['xtick.major.size'] = 15
plt.rcParams['ytick.major.size'] = 15

ax = sns.ecdfplot(data=omit_counts, x=omit_counts.num_omit_cookies, linewidth=8)
ax.set(xlim=(0, 350))

plt.xlabel('# Omitted Cookies/Website')
plt.ylabel('CDF')
# out_file = FIG_DIR / 'eu_omit_cookie_cdf.png'
# plt.savefig(out_file, dpi=200, bbox_inches='tight'); print(f"Written to {out_file}")

In [ ]:
plot_df = omit_counts[:50]

sns.set(rc={'figure.figsize':(24,8),"font.size":35,"axes.titlesize":35,"axes.labelsize":25},style="white", font_scale=1.5)
ax = sns.barplot(x=plot_df.index, y=plot_df.num_omit_cookies)
ticks = plt.xticks(rotation=45, ha='right')

ax.tick_params(left=True, bottom=False, direction="in")
ax.set_ylabel('# Omitted Cookies')
ax.set_xlabel('Website')

# out_file = FIG_DIR / 'omit_site.png'
# plt.savefig(out_file, dpi=200, bbox_inches='tight'); print(f"Written to {out_file}")

In [ ]:
def plot_histogram_omit():
    sns.set(rc={'figure.figsize':(9,6),"font.size":25,"axes.titlesize":25,"axes.labelsize":15},style="white", font_scale = 1)
    omit_counts_histo = omit_complies.site.value_counts().value_counts()
    sns.barplot(x=omit_counts_histo.index, y=omit_counts_histo.values)
    ticks = plt.xticks(rotation=45, ha='right')

## Plot distribution of incorrectly enforced cookies

In [ ]:
incor_complies = complies[complies.comply == 'incorrect']
# set(cookielist_sites) - set(omit_complies.site.unique())
incor_counts = incor_complies.site.value_counts().iloc[:50]
incor_counts = incor_counts.to_frame().reset_index().sort_values(by=['site', 'index'], ascending=[False, True])
incor_counts.columns

In [ ]:
sns.set(rc={'figure.figsize':(24,8),"font.size":35,"axes.titlesize":35,"axes.labelsize":25},style="white", font_scale=1.5)
ax = sns.barplot(x='index', y='site', data=incor_counts)

ticks = plt.xticks(rotation=45, ha='right')

ax.tick_params(left=True, bottom=True, direction="in")
ax.set_ylabel('# Incorrectly Enforced Cookies')
ax.set_xlabel('Website')

# out_file = FIG_DIR / 'incor_site.png'
# plt.savefig(out_file, dpi=200, bbox_inches='tight'); print(f"Written to {out_file}")

In [ ]:
# len(incor_counts)

In [ ]:
incor_counts[incor_counts.site == 33].reset_index(drop=True)

### Top incorrect cookies

In [ ]:
incor_cookies = complies[complies.comply == 'incorrect']

In [ ]:
# site = 'onetrust.com'
# complies[complies.site == site].reset_index(drop=True)
# cookie_prefs[(cookie_prefs.site == site) & (cookie_prefs.name == '_gid')]

In [ ]:
top_incor_cookies = incor_cookies.name.value_counts().sort_values(ascending=False).to_frame().reset_index().rename(columns={'index':'cookie_name', 'name': 'num_occur'})

In [ ]:
top_incor_cookies['num_sites'] = top_incor_cookies.cookie_name.map(lambda cookie_name: incor_cookies[incor_cookies.name == cookie_name].site.nunique())

In [ ]:
print(top_incor_cookies.head(5)[['cookie_name', 'num_sites']].rename(columns={'cookie_name': 'Cookie Name', 'num_sites': '# Websites'}).to_latex(index=False))

# Narrow cookie consent preferences

In [ ]:
consent_cookies = sent_cookies[sent_cookies.name.isin(['OptanonConsent', 'CookieConsent'])][['domain', 'name', 'site']].drop_duplicates()
narrow_consent_cookies = consent_cookies[~consent_cookies.domain.str.startswith('.')].reset_index(drop=True)

In [ ]:
set(contra_cookies.site).intersection(set(narrow_consent_cookies.site))

In [ ]:
narrow_consent_cookies

# Case studies

In [ ]:
incor_df = complies[complies.comply == 'incorrect']
incor_df.site.value_counts().sort_values(ascending=False).head()

In [ ]:
case_site = 'blabbermouth.net'
incor_df[incor_df.site == case_site]

In [ ]:
# cookie_prefs[(cookie_prefs.site == 'scientificamerican.com') & (cookie_prefs.name == 'MXP_TRACKINGID')]
cookie_prefs[(cookie_prefs.site == case_site) & (cookie_prefs.name == 'IDE')]

In [ ]:
# sent_cookies[(sent_cookies.site == 'scientificamerican.com') & (sent_cookies.name == 'MXP_TRACKINGID')]

In [ ]:
omit_df = complies[complies.comply == 'omit']
omit_df.site.value_counts().sort_values(ascending=False).iloc[20:30]

In [ ]:
print(omit_df[omit_df.site == 'scientificamerican.com'].reset_index().name.tolist())

In [ ]:
'uids' in omit_df[omit_df.site == 'scientificamerican.com'].reset_index().name.tolist()

In [ ]:
omit_df[(omit_df.site == 'scientificamerican.com') & (omit_df.name == 'cftoken')]

In [ ]:
cookie_prefs[(cookie_prefs.site == 'scientificamerican.com') & (cookie_prefs.name == 'ADGRX_UID')]

In [ ]:
all_sites = set(complies.site)
nothing_comply_sites = all_sites - set(complies[complies.comply == 'comply'].site)
print("Sites with no compliant cookies:", nothing_comply_sites)

In [ ]:
for site in ['liveperson.com']: # nothing_comply_sites:
    print(site)
site_cookies = postrej_br_cookies[postrej_br_cookies.site == site]

In [ ]:
acookie = site_cookies[site_cookies.name == 'OptanonAlertBoxClosed'].iloc[0]

In [ ]:
site_prefs = cookie_prefs[cookie_prefs.site == site]
consent_cookies = site_prefs[site_prefs.consent]
consent_cookie_names = consent_cookies.name
site_cookies[site_cookies.name.isin(consent_cookie_names)]
# cookie_name = 'OptanonAlertBoxClosed'
# cookie_pref = site_prefs[site_prefs.name == cookie_name].iloc[0]
# cookie_pref

In [ ]:
consent_cookies

In [ ]:
import sys; import importlib; importlib.reload(sys.modules['consent.consistency.cookie_pref_match'])
from consent.consistency.cookie_pref_match import cookie_pref_match
cookie_pref_match(acookie, cookie_pref, site, verbose=2)

In [ ]:
# Analyze Contras